In [1]:
import nltk 

In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
sid = SentimentIntensityAnalyzer()

In [7]:
a = 'This is a very not good movie'

In [8]:
sid.polarity_scores(a)

{'neg': 0.349, 'neu': 0.651, 'pos': 0.0, 'compound': -0.3988}

In [9]:
a = 'This was the best, most awesome movie EVER MADE!!!'

In [10]:
sid.polarity_scores(a)

{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

In [11]:
a = 'This was the worst movie thaat has ever been disgraced on screen'

In [12]:
sid.polarity_scores(a)

{'neg': 0.415, 'neu': 0.585, 'pos': 0.0, 'compound': -0.7964}

In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('amazonreviews.tsv',sep='\t')

In [15]:
df.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [16]:
df.isna().sum()

label     0
review    0
dtype: int64

In [17]:
df['label'].value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [18]:
df.dropna(inplace=True)

In [19]:
len(df)

10000

In [20]:
blanks = []
for i,lb,rv in df.itertuples():
    # (index,label, review)
    if type(rv) == str:
        if(rv.isspace()):
            blanks.append(i)

In [23]:
#df.drop(blanks,inplace=True)

In [24]:
df['review'][0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [25]:
sid.polarity_scores(df['review'][0])

{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'compound': 0.9454}

In [26]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))

In [27]:
df.head()

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."


In [28]:
df['compound'] = df['scores'].apply(lambda d:d['compound'])

In [30]:
df.head()

,label,review,scores,compound
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781


In [31]:
df['comp_score'] = df['compound'].apply(lambda score: 'pos' if score>=0 else 'neg')

In [32]:
df.head()

,label,review,scores,compound,comp_score
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


In [33]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

In [34]:
accuracy_score(df['label'],df['comp_score'])

0.7097

In [36]:
cm = confusion_matrix(df['label'],df['comp_score'])

In [38]:
conf_df = pd.DataFrame(cm,columns=['True Label','Predicted Label'])

In [39]:
conf_df

,True Label,Predicted Label
0,2629,2468
1,435,4468


In [40]:
print(classification_report(df['label'],df['comp_score']))

              precision    recall  f1-score   support

         neg       0.86      0.52      0.64      5097
         pos       0.64      0.91      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.71      0.70     10000
weighted avg       0.75      0.71      0.70     10000

